In [55]:
import pandas as pd
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import random

In [56]:
def optimization(h0,
                 invested,
                 r_vec,
                 var_vec,
                 p_vec,
                 com_rate,
                 risk_para,
                 trade_para):
    """
    Function purpose
    -------------------------------------
    Solves for a terminal portfolio value
    
    
    Parameters
    ----------
    h0 : float
        Starting value of asset in portfolio
    invested : float [0,1]
        Percent invested
    r_vec : list of floats
        Forecasted asset returns
    var_vec : list of floats
        Forecasted asset variances
    p_vec : list of floats
        Percentage of current portfolio invested
    com_rate : float
        Commission rate of executing a trade
    risk_para : float (>= 0)
        Risk aversion parameter
    trade_para : float (>=0)
        Trading aversion parameter
    """
    
    # Initialize portfolio values
    V = h0 # Total portfolio value
    asset_bal = V * invested # Amount invested in asset
    cash_bal = V - asset_bal # Amount invested in cash
    
    # Compute terminal portfolio value
    for i, r in enumerate(r_vec):
        if i == 0:
            amount_chg = V * (p_vec[i] - invested)
        else:
            amount_chg = V * (p_vec[i] - p_vec[i-1])
        
        asset_bal = (asset_bal + amount_chg) * (1 + r) # Update invested amount, grow by next period return
        cash_bal -= amount_chg + np.abs(amount_chg) * com_rate # Update cash balance (less cost of trade)
        V = asset_bal + cash_bal # Update portfolio balance
        risk_av = risk_para * (asset_bal * var_vec[i] * asset_bal) / V # Risk aversion
        trade_av = trade_para * np.abs(p_vec[i] - p_vec[i-1]) # Trade aversion
        V -= risk_av + trade_av

    return V

In [69]:
# Constraints on portfolio allocation (keep percentage allocated between 0 and 1)
cons = ({'type': 'ineq', 'fun': lambda x:  x - 1e-6}, # x[i] >= 0
        {'type': 'ineq', 'fun': lambda x:  (1 - 1e-6) - x}) # x[i] <= 1

In [70]:
h0 = 100
K = 100
com_rate = 0.01
risk_para = 100
trade_para = 5

r_vec = np.random.normal(0.01, .03, K)
var_vec = np.random.normal(0.005,0.005,size=K)**2
p_vec = np.random.uniform(size=K)

In [71]:
optimization(100,
             1,
             r_vec,
             var_vec,
             p_vec,
             com_rate=com_rate,
             risk_para=risk_para,
             trade_para=trade_para)

126.41311891767842

In [72]:
prob = optimize.minimize(lambda x: -optimization(100,
                                                 1,
                                                 r_vec,
                                                 var_vec,
                                                 x,
                                                 com_rate=com_rate,
                                                 risk_para=risk_para,
                                                 trade_para=trade_para),
                         np.random.uniform(size=K),
                         method='SLSQP',
                         constraints=cons)

In [73]:
np.max(prob.x), np.min(prob.x)

(0.9999990024047859, 9.96091464525762e-07)

In [74]:
prob.fun

-378.6036967792329

In [75]:
prob.x

array([9.99999002e-01, 1.00034658e-06, 9.99731165e-07, 9.97434336e-07,
       9.97779167e-07, 9.97713216e-07, 9.99999001e-01, 9.99428616e-07,
       9.99004109e-07, 9.99999001e-01, 9.91823137e-01, 9.96034878e-01,
       9.99999002e-01, 9.99998990e-01, 9.99999001e-01, 9.99999001e-01,
       9.98538979e-07, 9.99067904e-07, 9.99999001e-01, 9.99999001e-01,
       9.99999002e-01, 9.99999002e-01, 9.99999001e-01, 9.98961246e-07,
       9.99999001e-01, 9.98972952e-07, 9.98444739e-07, 9.99999002e-01,
       9.99283514e-07, 9.98089429e-07, 9.99999000e-01, 3.17671233e-01,
       9.98966570e-07, 9.99999002e-01, 9.99999001e-01, 9.99999002e-01,
       9.99999001e-01, 9.99999001e-01, 9.98750526e-07, 9.97761889e-07,
       9.99999000e-01, 9.99999001e-01, 3.19912025e-04, 9.96091465e-07,
       9.99999000e-01, 9.99999001e-01, 9.99999002e-01, 9.98167685e-01,
       9.99932685e-07, 9.99999001e-01, 9.99999002e-01, 9.99999002e-01,
       9.98594579e-07, 9.99999002e-01, 9.93873450e-01, 9.99999001e-01,
      

In [76]:
p_vec = prob.x

In [77]:
optimization(100,
             1,
             r_vec,
             var_vec,
             p_vec,
             com_rate=com_rate,
             risk_para=risk_para,
             trade_para=trade_para)

378.6036967792329

In [78]:
### Test using sample returns from dataset

In [79]:
data = pd.read_csv('~/Desktop/Corbin SBU/AMS 516/Project/Code/MSCI_Data.csv')

data.Date = pd.to_datetime(data.Date, format='%m/%d/%y')

data['log_returns'] = np.log(data.Close) - np.log(data.Close.shift(1))

Neff = 260
Volatility = np.concatenate((np.zeros(Neff),
                             [np.sqrt(sum((data['log_returns'][i:Neff+i] -
                                           np.mean(data['log_returns'][i:Neff+i]))**2) / Neff)
                              for i in range(len(data)-Neff)]))
data['Volatility'] = Volatility

In [80]:
data

,Date,Close,log_returns,Volatility
0,1999-01-01,1149.952,NaN,0.000000
1,1999-01-04,1161.524,0.010013,0.000000
2,1999-01-05,1173.340,0.010121,0.000000
3,1999-01-06,1195.452,0.018670,0.000000
4,1999-01-07,1191.633,-0.003200,0.000000
...,...,...,...,...
4430,2015-12-25,1673.299,-0.000418,0.008154
4431,2015-12-28,1670.387,-0.001742,0.008152
4432,2015-12-29,1685.618,0.009077,0.008152
4433,2015-12-30,1676.432,-0.005465,0.008164


In [86]:
h0 = 100
invested = .75
K = 100
com_rate = 0.001
risk_para = 2
trade_para = 0.02

# Randomly grab returns
num = np.random.randint(Neff+1, high = len(data) - 1 - 100)
r_vec = data.log_returns[num:num + K].values
var_vec = data.Volatility[num:num + K].values**2
p_vec = np.random.uniform(size=K)

print(optimization(h0,
                   invested,
                   r_vec,
                   var_vec,
                   p_vec,
                   com_rate,
                   risk_para,
                   trade_para))

prob = optimize.minimize(lambda x: -optimization(h0,
                                                 invested,
                                                 r_vec,
                                                 var_vec,
                                                 x,
                                                 com_rate,
                                                 risk_para,
                                                 trade_para),
                         np.random.uniform(size=K),
                         method='SLSQP',
                         constraints=cons)
p_vec = prob.x

print(optimization(h0,
                   invested,
                   r_vec,
                   var_vec,
                   p_vec,
                   com_rate,
                   risk_para,
                   trade_para))

print(f'First trade: Change investment allocation from {invested} to {np.abs(np.round(prob.x[0], 6)):.02f}')

print(prob.x[0])

96.2568556226023
133.4934431339204
First trade: Change investment allocation from 0.75 to 1.00
0.9999990000079344
